In [1]:
from utils import CLASSES, load_voc, convert
from dataset import Articles
from model import TopicClassifier
import torch
import random
import ast


In [2]:
def load(name, vocabulary, CLASSES):
    model = TopicClassifier(vocabulary, len(CLASSES))
    model.load_state_dict(torch.load(name + ".pth")["state_dict"])
    model.cuda()
    model.eval()
    model.train(False)
    return model


In [3]:
vocabulary = load_voc()


valid = Articles("valid.csv", len(CLASSES))
# print(len(valid))

# for i in range(0, 130, 10):
#     model = load(str(120-i) + "_topic", len(vocabulary), CLASSES)

model = load("topicClassifier", len(vocabulary), CLASSES)
output_size = 50  # depend on the model
k = 0
for j in range(len(valid)):
    text, topic = valid.__getitem__(j)
    y = model(text.view(1, output_size).cuda())
    _, predicted = torch.max(y.data, 1)
    if CLASSES[predicted.item()] == CLASSES[topic.item()]:
        k += 1
print("accuracy: " + str(float(k / len(valid) * 100)) + " %")


In [4]:
topic = "78.10Z" #the topic of the example
#the text of the example
text = convert(vocabulary,"Ipsen a signé un partenariat de recherche avec PeptiMimesis, une jeune pousse issue de l'INSERM et de l'Université de Strasbourg. L'accord comprend une option de licence pour le développement et la commercialisation de nouveaux peptides thérapeutiques dans le domaine de l'oncologie. Ipsen et PeptiMimesis ont prévu de travailler en 'étroite collaboration'. La société alsacienne recevra le soutien financier d'Ipsen pour les phases de recherche ainsi que des paiements d'étapes liés aux avancées sur ce programme. En cas d'exercice de l'option, Ipsen pourrait en outre verser à PeptiMimesis des paiements potentiels liés à l'atteinte d'étapes de développement, réglementaires et commerciales ainsi que des redevances sur les ventes nettes mondiales annuelles. Les détails financiers de l'accord n'ont pas été dévoilés.")
text = torch.LongTensor(text[:50])
length = list(text.size())[0]
print(length)
y = model(text.view(1, length).cuda())
_, predicted = torch.max(y.data, 1)
print(str(CLASSES[predicted.item()])+": "+topic)